In [1]:
import tensorflow as tf
from reader import create_inputs
import json
import os
from model import RecogModel

In [2]:
param_filename = "recog_params.json"
with open(param_filename, 'r') as f:
        recog_params = json.load(f)

In [3]:
image, label_value, label_shape, label_index = create_inputs(input_channel=1,
                                                             labels=recog_params['labels'],
                                                             dilations=recog_params['ctc_params']['cnn']['dilations'])

queue = tf.FIFOQueue(256, ['uint8', 'uint8', 'uint8', 'uint8'])
enqueue = queue.enqueue([image, label_value, label_shape, label_index])
input_data = queue.dequeue()

In [4]:
net = RecogModel(input_channel=1,
                klass=len(recog_params['labels']),
                batch_size=1,
                network_type=recog_params['network_type'],
                ctc_params=recog_params['ctc_params'],
                seq2seq_params=recog_params['seq2seq_params'])

image, label_value, label_shape, label_index = net._preprocess(input_data)
output = net._create_network(image)

sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

coord = tf.train.Coordinator()
qr = tf.train.QueueRunner(queue, [enqueue])
qr.create_threads(sess, coord=coord, start=True)
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [5]:
output = tf.reshape(output, [net.batch_size, -1, net.klass + 1])
sparse_label = tf.SparseTensor(label_index, label_value, label_shape)
loss = tf.nn.ctc_loss(inputs=output,
                      labels=sparse_label,
                      sequence_length=tf.expand_dims(tf.shape(output)[1], 0),
                      time_major=False)
reduced_loss = tf.reduce_mean(loss)
output_data, loss_data, reduced_loss_data = sess.run([output, loss, reduced_loss])

In [6]:
output_data.shape
loss_data
reduced_loss_data

125.53495